# <center>**Analyse exploratoire des donnees**

In [1]:
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import plotly_express as px
import json

import polars as pl

In [2]:
# Définir les informations de connexion à la base de données
with open('config.json', 'r') as f:
    config = json.load(f)

db_user = config['db_user']
db_password = config['db_password']
db_host = config['db_host']
db_port = config['db_port']
db_name = config['db_name']

connection_string = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

# Create database engine
engine = create_engine(connection_string)

## Analyse des tentances temporelles des trajets en taxi au fil des mois, des jours de la semaine et des heures

Toutes les analyses seront faites sur l'annee complete la plus recente soit 2023.

In [4]:
# Extraction des données des trajets en taxi
query = """
    SELECT tpep_pickup_datetime, COUNT(*) AS num_trips
    FROM ytaxi_histo
    WHERE 
	(trip_distance > 0 AND passenger_count > 0) 
	AND 
	(EXTRACT(YEAR FROM tpep_pickup_datetime) = 2023 AND EXTRACT(YEAR FROM tpep_dropoff_datetime) = 2023)
    AND total_amount > 0
    GROUP BY tpep_pickup_datetime;
"""
df = pd.read_sql(query, engine)
df.head()

,tpep_pickup_datetime,num_trips
0,2023-01-01 00:00:05,1
1,2023-01-01 00:00:06,1
2,2023-01-01 00:00:08,1
3,2023-01-01 00:00:09,1
4,2023-01-01 00:00:13,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18842758 entries, 0 to 18842757
Data columns (total 2 columns):
 #   Column                Dtype         
---  ------                -----         
 0   tpep_pickup_datetime  datetime64[ns]
 1   num_trips             int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 287.5 MB


In [6]:
# Analyse des tendances temporelles au fil des mois
fig_monthly = px.line(df.resample('ME', on='tpep_pickup_datetime').sum(), 
                      x=df.resample('ME', on='tpep_pickup_datetime').sum().index, 
                      y='num_trips', 
                      title='Tendances mensuelles des trajets en taxi')
fig_monthly.update_xaxes(title='Mois')
fig_monthly.update_yaxes(title='Nombre total de trajets')

# Analyse des tendances temporelles au fil des jours de la semaine
fig_weekly = px.line(df.resample('D', on='tpep_pickup_datetime').sum(), 
                     x=df.resample('D', on='tpep_pickup_datetime').sum().index, 
                     y='num_trips', 
                     title='Tendances journalières des trajets en taxi')
fig_weekly.update_xaxes(title='Jour de la semaine')
fig_weekly.update_yaxes(title='Nombre total de trajets')

# Analyse des tendances temporelles au fil des heures de la journée
fig_hourly = px.line(df.resample('h', on='tpep_pickup_datetime').sum(), 
                     x=df.resample('h', on='tpep_pickup_datetime').sum().index, 
                     y='num_trips', 
                     title='Tendances horaires des trajets en taxi')
fig_hourly.update_xaxes(title='Heure de la journée')
fig_hourly.update_yaxes(title='Nombre total de trajets')

# Affichage des résultats
fig_monthly.show()
fig_weekly.show()
fig_hourly.show()


In [7]:
# Extraire le mois à partir de tpep_pickup_datetime
df['month'] = df['tpep_pickup_datetime'].dt.month

# Agréger les données en fonction du mois
df_monthly = df.groupby('month')['num_trips'].sum().reset_index()

# Afficher la DataFrame résultante
df_monthly

,month,num_trips
0,1,2884431
1,2,2732837
2,3,3190910
3,4,3078075
4,5,3282018
5,6,3085063
6,7,2713181
7,8,2629087
8,9,2605041
9,10,3243964


In [8]:
# Mapper les numéros de mois aux noms des mois
month_names = {
    1: "January",
    2: "February",
    3: "March",
    4: "April",
    5: "May",
    6: "June",
    7: "July",
    8: "August",
    9: "September",
    10: "October",
    11: "November",
    12: "December"
}

df_monthly['Month'] = df_monthly['month'].map(month_names)

# Tracez le graphique de barres
fig = px.bar(df_monthly, x="Month", y="num_trips", 
             labels={"Month": "Month", "Num_Trips": "Average Number of Trips"})
fig.update_layout(title="Total of Taxi Trips per Month")
fig.show()

In [9]:
# Extraire le jour de la semaine à partir de tpep_pickup_datetime (0 pour lundi, 1 pour mardi, ..., 6 pour dimanche)
df['day_of_week'] = df['tpep_pickup_datetime'].dt.dayofweek

# Agréger les données en fonction du jour de la semaine
df_weekly = df.groupby('day_of_week')['num_trips'].sum().reset_index()

# Afficher la DataFrame résultante
df_weekly

,day_of_week,num_trips
0,0,4449071
1,1,5175870
2,2,5480227
3,3,5591866
4,4,5282888
5,5,5167929
6,6,4465779


In [10]:
# Mapper les numéros de jour de la semaine aux noms des jours de la semaine
day_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday"
}

df_weekly['Day_of_week'] = df_weekly['day_of_week'].map(day_names)

# Tracez le graphique de barres
fig = px.bar(df_weekly, x="Day_of_week", y="num_trips", 
             labels={"Day_of_week": "Day of Week", "Num_Trips": "Average Number of Trips"})
fig.update_layout(title="Total Number of Taxi Trips per Day of Week")
fig.show()

In [11]:
# Créez une fonction pour vérifier si le jour de la semaine est un week-end
def is_weekend(day):
    return day in [5, 6]  # 5 correspond à samedi et 6 correspond à dimanche

# Appliquez la fonction à la colonne "Day_of_week" pour créer la nouvelle colonne booléenne "is_weekend"
df_weekly['is_weekend'] = df_weekly['day_of_week'].apply(lambda x: is_weekend(x))

# Remplacer les valeurs booléennes par des étiquettes explicites
df_weekly['is_weekend'] = df_weekly['is_weekend'].replace({True: 'Weekend', False: 'Week'})

# Affichez le DataFrame avec la nouvelle colonne
df_weekly

,day_of_week,num_trips,Day_of_week,is_weekend
0,0,4449071,Monday,Week
1,1,5175870,Tuesday,Week
2,2,5480227,Wednesday,Week
3,3,5591866,Thursday,Week
4,4,5282888,Friday,Week
5,5,5167929,Saturday,Weekend
6,6,4465779,Sunday,Weekend


In [12]:
# Groupez les données selon la colonne "is_weekend" et faites la somme de "num_trips"
df_grouped = df_weekly.groupby('is_weekend')['num_trips'].sum().reset_index()

# Tracez le Pie Chart
fig = px.pie(df_grouped, values='num_trips', names='is_weekend', title='Percentage of Taxi Trips on Weekends')
fig.show()

In [13]:
# Extraire l'heure à partir de tpep_pickup_datetime
df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour

# Agréger les données en fonction de l'heure de la journée
df_hourly = df.groupby('pickup_hour')['num_trips'].sum().reset_index()

# Afficher la DataFrame résultante
df_hourly

,pickup_hour,num_trips
0,0,996564
1,1,667002
2,2,437286
3,3,284314
4,4,181610
5,5,194991
6,6,475997
7,7,947428
8,8,1323115
9,9,1516191


In [14]:
# Tracez le graphique de barres
fig = px.bar(df_hourly, x="pickup_hour", y="num_trips")
fig.update_layout(title="Average Number of Taxi Trips per Hour of the day")
fig.show()

Ce graphique représente le nombre de voyages (num_trips) effectués au cours de chaque heure de la journée (pickup_hour) dans un service de taxi jaune à New York. Voici une brève analyse de ces données :

- Les heures de pointe sont entre 8h et 18h, avec un pic vers 18h.
- Les heures creuses sont entre minuit et 6h du matin.
- Les chiffres sont cohérents avec les habitudes de déplacement dans une grande ville, où le volume de voyages tend à augmenter pendant les heures de travail et à diminuer pendant la nuit.

Ce tableau pourrait être utilisé pour planifier les opérations de taxi, allouer des ressources en conséquence et analyser les tendances de demande de services de taxi au fil du temps.

## Analyse des zones

In [16]:
df_zones = pd.read_sql("SELECT * FROM taxi_zone;", engine)
df_zones

,locationid,borough,zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,None,None


In [17]:
# Charger uniquement les colonnes pertinentes de df_zones pour réduire la mémoire utilisée
df_zones_subset = df_zones[['locationid', 'borough', 'zone', 'service_zone']]

# Extraction des données des trajets en taxi de l'annee 2023
query = """
    SELECT *
    FROM ytaxi_histo
    WHERE 
	(trip_distance > 0 AND passenger_count > 0 AND total_amount > 0) 
	AND 
	(EXTRACT(YEAR FROM tpep_pickup_datetime) = 2023 AND EXTRACT(YEAR FROM tpep_dropoff_datetime) = 2023);
"""

# Définir la taille des morceaux pour la fusion
chunksize = 100000  # par exemple, fusionner par paquets de 100 milles lignes

# Generateur des morceaux de la table ytaxi_histo
result_generator = pd.read_sql(query, engine, chunksize=chunksize)

# Initialiser une liste pour stocker les morceaux fusionnés
merged_chunks = []

# Parcourir les morceaux de df_trips
for chunk in result_generator:
    # Fusionner le morceau actuel avec df_zones_subset sur pulocationid
    merged_chunk = pd.merge(chunk, df_zones_subset.rename(columns={'borough': 'puborough', 'zone': 'puzone', 'service_zone': 'pu_service_zone'}),
                            how='left', left_on='pulocationid', right_on='locationid')
    
    # Fusionner le morceau actuel avec df_zones_subset sur dolocationid
    merged_chunk = pd.merge(merged_chunk, df_zones_subset.rename(columns={'borough': 'doborough', 'zone': 'dozone', 'service_zone': 'do_service_zone'}),
                            how='left', left_on='dolocationid', right_on='locationid')
    
    # Ajouter le morceau fusionné à la liste
    merged_chunks.append(merged_chunk)

# Concaténer tous les morceaux fusionnés pour former la DataFrame finale
df_trips = pd.concat(merged_chunks, ignore_index=True)

# Supprimer les colonnes redondantes
df_trips.drop(columns=['locationid_x', 'locationid_y'], inplace=True)

# Afficher la DataFrame avec les nouvelles colonnes
df_trips.head()

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,...,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,puborough,puzone,pu_service_zone,doborough,dozone,do_service_zone
0,2,2023-08-02 14:58:35,2023-08-02 15:39:09,1.0,14.59,1.0,N,132,226,2,...,1.0,61.55,0.0,1.75,Queens,JFK Airport,Airports,Queens,Sunnyside,Boro Zone
1,2,2023-08-01 14:35:26,2023-08-01 14:55:15,1.0,2.76,1.0,N,137,262,1,...,1.0,24.64,2.5,0.00,Manhattan,Kips Bay,Yellow Zone,Manhattan,Yorkville East,Yellow Zone
2,1,2023-08-01 14:47:04,2023-08-01 15:15:28,1.0,3.50,1.0,N,231,48,1,...,1.0,32.75,2.5,0.00,Manhattan,TriBeCa/Civic Center,Yellow Zone,Manhattan,Clinton East,Yellow Zone
3,2,2023-08-01 14:27:50,2023-08-01 14:47:58,1.0,3.08,1.0,N,107,140,2,...,1.0,23.80,2.5,0.00,Manhattan,Gramercy,Yellow Zone,Manhattan,Lenox Hill East,Yellow Zone
4,2,2023-08-01 14:55:52,2023-08-01 15:23:06,1.0,9.98,1.0,N,140,138,1,...,1.0,66.78,2.5,0.00,Manhattan,Lenox Hill East,Yellow Zone,Queens,LaGuardia Airport,Airports


1. `df_zones = pd.read_sql("SELECT * FROM taxi_zone;", engine)`: Cette ligne lit les données de la table "taxi_zone" depuis une base de données en utilisant la connexion `engine` et stocke ces données dans un DataFrame pandas appelé `df_zones`.

2. `df_zones_subset = df_zones[['locationid', 'borough', 'zone', 'service_zone']]`: Cette ligne crée un nouveau DataFrame appelé `df_zones_subset` en sélectionnant uniquement les colonnes pertinentes ('locationid', 'borough', 'zone', 'service_zone') à partir du DataFrame `df_zones`. Cela réduit la mémoire utilisée en ne conservant que les colonnes nécessaires.

3. `query = """SELECT * FROM ytaxi_histo WHERE (trip_distance > 0 AND passenger_count > 0 AND total_amount > 0) AND (EXTRACT(YEAR FROM tpep_pickup_datetime) = 2023 AND EXTRACT(YEAR FROM tpep_dropoff_datetime) = 2023);"""`: Cette requête SQL extrait les données des trajets en taxi de l'année 2023 de la table `ytaxi_histo` en filtrant les lignes selon les conditions spécifiées (distance de trajet, nombre de passagers et montant total positifs, et année de prise en charge et de dépose).

4. `chunksize = 100000`: Cette ligne définit la taille des morceaux (ou chunks) pour la fusion des données, ce qui permet de traiter les données par lots plus gérables.

5. `result_generator = pd.read_sql(query, engine, chunksize=chunksize)`: Cette ligne utilise `pd.read_sql` pour exécuter la requête SQL définie dans `query` et lire les résultats de manière itérative en morceaux (chunks) de taille `chunksize`. Cela évite de charger toutes les données en mémoire à la fois.

6. `merged_chunks = []`: Cette ligne initialise une liste vide `merged_chunks` qui sera utilisée pour stocker les morceaux fusionnés des données.

7. La boucle `for chunk in result_generator:` itère sur chaque morceau de données extrait par le générateur `result_generator`.

8. `merged_chunk = pd.merge(chunk, df_zones_subset.rename(columns={'borough': 'puborough', 'zone': 'puzone', 'service_zone': 'pu_service_zone'}), how='left', left_on='pulocationid', right_on='locationid')`: Cette ligne fusionne le morceau actuel (`chunk`) avec le DataFrame `df_zones_subset` sur la colonne `pulocationid`, en renommant les colonnes pertinentes de `df_zones_subset` selon les spécifications indiquées.

9. `merged_chunk = pd.merge(merged_chunk, df_zones_subset.rename(columns={'borough': 'doborough', 'zone': 'dozone', 'service_zone': 'do_service_zone'}), how='left', left_on='dolocationid', right_on='locationid')`: Cette ligne fusionne le morceau actuel (`merged_chunk`) avec le DataFrame `df_zones_subset` sur la colonne `dolocationid`, en renommant les colonnes pertinentes de `df_zones_subset` selon les spécifications indiquées.

10. `merged_chunks.append(merged_chunk)`: Cette ligne ajoute le morceau fusionné (`merged_chunk`) à la liste `merged_chunks`.

11. `df_trips = pd.concat(merged_chunks, ignore_index=True)`: Cette ligne concatène tous les morceaux fusionnés stockés dans la liste `merged_chunks` pour former le DataFrame final `df_trips`, en ignorant les index d'origine.

12. `df_trips.drop(columns=['locationid_x', 'locationid_y'], inplace=True)`: Cette ligne supprime les colonnes redondantes 'locationid_x' et 'locationid_y' de `df_trips` afin d'éliminer les doublons résultant de la fusion.

13. `df_trips.head()`: Cette ligne affiche les premières lignes du DataFrame `df_trips` avec les nouvelles colonnes fusionnées.

In [18]:
df_trips.isna().sum()

vendorid                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count               0
trip_distance                 0
ratecodeid                    0
store_and_fwd_flag            0
pulocationid                  0
dolocationid                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge          0
airport_fee                   0
puborough                  6512
puzone                   313885
pu_service_zone          320397
doborough                142141
dozone                   322600
do_service_zone          464741
dtype: int64

Ce résultat de `df_trips.isna().sum()` indique le nombre de valeurs manquantes (NaN) pour chaque colonne de la DataFrame `df_trips`. Voici ce que chaque colonne représente :

- `puborough`: Le nombre de valeurs manquantes dans cette colonne indique le nombre de trajets où la zone de prise en charge (`pulocationid`) n'a pas de correspondance dans la DataFrame `df_zones` pour le nom du borough.
- `puzone`: Le nombre de valeurs manquantes dans cette colonne indique le nombre de trajets où la zone de prise en charge (`pulocationid`) n'a pas de correspondance dans la DataFrame `df_zones` pour le nom de la zone.
- `pu_service_zone`: Le nombre de valeurs manquantes dans cette colonne indique le nombre de trajets où la zone de prise en charge (`pulocationid`) n'a pas de correspondance dans la DataFrame `df_zones` pour la zone de service.
- `doborough`: Le nombre de valeurs manquantes dans cette colonne indique le nombre de trajets où la zone de dépose (`dolocationid`) n'a pas de correspondance dans la DataFrame `df_zones` pour le nom du borough.
- `dozone`: Le nombre de valeurs manquantes dans cette colonne indique le nombre de trajets où la zone de dépose (`dolocationid`) n'a pas de correspondance dans la DataFrame `df_zones` pour le nom de la zone.
- `do_service_zone`: Le nombre de valeurs manquantes dans cette colonne indique le nombre de trajets où la zone de dépose (`dolocationid`) n'a pas de correspondance dans la DataFrame `df_zones` pour la zone de service.

En résumé, les valeurs manquantes dans ces colonnes indiquent des incohérences ou des données manquantes dans la correspondance entre les identifiants de localisation (`pulocationid` et `dolocationid`) dans la DataFrame `df_trips` et les données de la DataFrame `df_zones`. Il peut être nécessaire d'examiner de plus près ces données manquantes pour comprendre pourquoi elles se produisent et comment les traiter en conséquence.

In [19]:
# Supprimer les lignes avec des valeurs manquantes dans les donnees
df_trips = df_trips.dropna(axis=0)

# Enregistrer la DataFrame nettoyée au format Parquet
df_trips.to_parquet('trips_cleaned.parquet', index=False)

In [20]:
print(df_trips.shape)
df_trips.isna().sum()

(35081996, 25)


vendorid                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
ratecodeid               0
store_and_fwd_flag       0
pulocationid             0
dolocationid             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
congestion_surcharge     0
airport_fee              0
puborough                0
puzone                   0
pu_service_zone          0
doborough                0
dozone                   0
do_service_zone          0
dtype: int64

### Arrondissements de prise en charge les plus populaires

In [21]:
# Compter le nombre de trajets par arrondissement de prise en charge
pickup_counts = df_trips['puborough'].value_counts().reset_index()
pickup_counts.columns = ['Arrondissement', 'Nombre de trajets']
pickup_counts

,Arrondissement,Nombre de trajets
0,Manhattan,31444556
1,Queens,3405732
2,Brooklyn,181620
3,Bronx,47487
4,Staten Island,1715
5,EWR,886


In [22]:
# Créer le graphique des arrondissements de prise en charge
fig = px.bar(pickup_counts, x='Arrondissement', y='Nombre de trajets', 
             title='Arrondissements de prise en charge',
             labels={'Arrondissement': 'Arrondissement de prise en charge', 'Nombre de trajets': 'Nombre de trajets'})

# Personnaliser la mise en page
fig.update_layout(xaxis_tickangle=-45, xaxis_tickfont=dict(size=10))

# Afficher le graphique
fig.show()

### Arrondissements de depose les plus populaires

In [23]:
# Compter le nombre de trajets par arrondissement de dépose
dropoff_counts = df_trips['doborough'].value_counts().reset_index()
dropoff_counts.columns = ['Arrondissement', 'Nombre de trajets']
print(dropoff_counts)

# Créer le graphique des arrondissements de dépose
fig = px.bar(dropoff_counts, x='Arrondissement', y='Nombre de trajets', 
             title='Arrondissements de dépose',
             labels={'Arrondissement': 'Arrondissement de dépose', 'Nombre de trajets': 'Nombre de trajets'})

# Personnaliser la mise en page
fig.update_layout(xaxis_tickangle=-45, xaxis_tickfont=dict(size=10))

# Afficher le graphique
fig.show()

  Arrondissement  Nombre de trajets
0      Manhattan           31598824
1         Queens            1807919
2       Brooklyn            1360202
3          Bronx             202558
4            EWR             102692
5  Staten Island               9801


ce graphique présente une vue agrégée du nombre de trajets de taxi dans différents arrondissements, ce qui pourrait fournir des informations utiles sur les tendances de déplacement dans la région.

### Itinéraires les plus populaires entre arrondissements

In [25]:
# Compter le nombre de trajets par paire d'arrondissements
route_counts = df_trips.groupby(['puborough', 'doborough']).size().reset_index(name='Nombre de trajets')
# Créer une nouvelle colonne par concaténation de puborough et doborough
route_counts['Itineraire'] = route_counts.apply(lambda row: f"{row['puborough']} - {row['doborough']}", axis=1)
route_counts = route_counts.sort_values(by='Nombre de trajets', ascending=False)
route_counts

,puborough,doborough,Nombre de trajets,Itineraire
21,Manhattan,Manhattan,29454540,Manhattan - Manhattan
27,Queens,Manhattan,2064689,Queens - Manhattan
22,Manhattan,Queens,1064815,Manhattan - Queens
19,Manhattan,Brooklyn,725531,Manhattan - Brooklyn
28,Queens,Queens,721952,Queens - Queens
25,Queens,Brooklyn,533767,Queens - Brooklyn
18,Manhattan,Bronx,99847,Manhattan - Bronx
7,Brooklyn,Brooklyn,96816,Brooklyn - Brooklyn
20,Manhattan,EWR,96105,Manhattan - EWR
24,Queens,Bronx,75097,Queens - Bronx


Ce résultat est un résumé du nombre de trajets de taxi effectués entre différents arrondissements, avec un décompte du nombre de trajets pour chaque itinéraire spécifique.

- **puborough**: C'est l'arrondissement de départ du trajet.
- **doborough**: C'est l'arrondissement de destination du trajet.
- **Nombre de trajets**: C'est le nombre total de trajets effectués entre l'arrondissement de départ et celui de destination.
- **Itineraire**: C'est une représentation textuelle de l'itinéraire, indiquant l'arrondissement de départ suivi de l'arrondissement de destination.

Par exemple, la première ligne indique qu'il y a eu 29 454 540 trajets de taxi partant de Manhattan et arrivant à Manhattan. La deuxième ligne montre qu'il y a eu 2 064 689 trajets de Queens à Manhattan, et ainsi de suite.

Ce tableau fournit une vue détaillée des flux de trafic entre différents arrondissements, ce qui pourrait être utile pour analyser les tendances de déplacement dans la région et pour la planification des transports.

In [26]:
# Top 5 des itineraires les plus populaires entre arrondissements
top5_route_counts = route_counts.head()

# Créer le graphique à barres horizontales
fig = px.bar(top5_route_counts, x='Itineraire', y='Nombre de trajets',
             title='Itinéraires les plus populaires entre arrondissements'
             )

# Afficher le graphique
fig.show()

### Zones de prise en charge

In [27]:
# Compter le nombre de trajets par zone de prise en charge
pickup_counts = df_trips['puzone'].value_counts().reset_index()
pickup_counts.columns = ['Zone', 'Nombre de trajets']
pickup_counts = pickup_counts.sort_values(by='Nombre de trajets', ascending=False)
print(pickup_counts)

# Créer le graphique des 20 zones de prise en charge les plus populaires
top20_pickup_counts = pickup_counts.head(20)

fig = px.bar(top20_pickup_counts, x='Nombre de trajets', y='Zone', 
             orientation='h',
             title='Top 20 des Zones de prise en charge les plus populaires',
             labels={'Zone': 'Zone de prise en charge', 'Nombre de trajets': 'Nombre de trajets'})

# Personnaliser la mise en page
fig.update_layout(xaxis_tickangle=-45, xaxis_tickfont=dict(size=10))

# Afficher le graphique
fig.show()

                      Zone  Nombre de trajets
0              JFK Airport            1816631
1    Upper East Side South            1698411
2           Midtown Center            1670629
3    Upper East Side North            1497825
4             Midtown East            1287226
..                     ...                ...
254    Grymes Hill/Clifton                  9
255        Freshkills Park                  7
256          West Brighton                  7
257          Port Richmond                  6
258                Oakwood                  3

[259 rows x 2 columns]


### Zones de deposes

In [28]:
# Compter le nombre de trajets par zone de prise en charge
dropoff_counts = df_trips['dozone'].value_counts().reset_index()
dropoff_counts.columns = ['Zone', 'Nombre de trajets']
dropoff_counts = dropoff_counts.sort_values(by='Nombre de trajets', ascending=False)
print(dropoff_counts)

# Créer le graphique des 20 zones de depose les plus populaires
top20_dropoff_counts = dropoff_counts.head(20)

fig = px.bar(top20_dropoff_counts, x='Nombre de trajets', y='Zone', 
             orientation='h',
             title='Top 20 des Zones de depose les plus populaires',
             labels={'Zone': 'Zone de depose', 'Nombre de trajets': 'Nombre de trajets'})

# Personnaliser la mise en page
fig.update_layout(xaxis_tickangle=-45, xaxis_tickfont=dict(size=10))

# Afficher le graphique
fig.show()

                                              Zone  Nombre de trajets
0                            Upper East Side North            1584454
1                            Upper East Side South            1517699
2                                   Midtown Center            1392343
3                        Times Sq/Theatre District            1101067
4                                      Murray Hill            1061509
..                                             ...                ...
254                              Rossville/Woodrow                173
255                                    Jamaica Bay                 69
256                                Freshkills Park                 36
257  Governor's Island/Ellis Island/Liberty Island                 17
258                               Great Kills Park                  2

[259 rows x 2 columns]


### Itineraires les plus populaires entre zones

In [29]:
# Compter le nombre de trajets par paire de zones
route_counts_zones = df_trips.groupby(['puzone', 'dozone']).size().reset_index(name='Nombre de trajets')

# Créer une nouvelle colonne par concaténation de puzone et dozone
route_counts_zones['Itinéraire'] = route_counts_zones.apply(lambda row: f"{row['puzone']} TO {row['dozone']}", axis=1)

# Trier par nombre de trajets
route_counts_zones = route_counts_zones.sort_values(by='Nombre de trajets', ascending=False)
print(route_counts_zones)

# Top 20 des itinéraires les plus populaires entre zones
top20_route_counts_zones = route_counts_zones.head(20)

# Créer le graphique à barres horizontales
fig = px.bar(top20_route_counts_zones, x='Itinéraire', y='Nombre de trajets',
             title='Top 20 Itinéraires les plus populaires entre zones'
             )

# Afficher le graphique
fig.show()

                        puzone                      dozone  Nombre de trajets  \
37104    Upper East Side South       Upper East Side North             247707   
36858    Upper East Side North       Upper East Side South             211151   
37105    Upper East Side South       Upper East Side South             165233   
36857    Upper East Side North       Upper East Side North             158612   
24714           Midtown Center       Upper East Side South             116764   
...                        ...                         ...                ...   
19313                  Jamaica      Marine Park/Mill Basin                  1   
3466   Briarwood/Jamaica Hills                Astoria Park                  1   
19309                  Jamaica             Manhattan Beach                  1   
19308                  Jamaica                     Madison                  1   
4690            Bushwick North  Spuyten Duyvil/Kingsbridge                  1   

                           

## Analyses sur le nombre de passagers

In [30]:
# Définir la taille du lot pour l'itération
chunk_size = 100000

# Lire le fichier parquet et obtenir les métadonnées
parquet_file = pq.ParquetFile('trips_cleaned.parquet')

# Initialiser une liste pour stocker les résultats partiels
partial_results = []

# Itérer sur les segments parquet par petits morceaux
for i in range(0, parquet_file.num_row_groups, chunk_size):
    # Lire un segment parquet
    chunk = parquet_file.read_row_group(i, columns=['puborough', 'doborough', 'passenger_count'])
    # Convertir le segment en DataFrame pandas
    chunk_df = chunk.to_pandas()
    # Créer une nouvelle colonne par concaténation de puborough et doborough pour chaque morceau
    chunk_df['Itineraire Arrondissement'] = chunk_df.apply(lambda row: f"{row['puborough']} - {row['doborough']}", axis=1)
    # Regrouper par Itineraire pour chaque morceau et calculer la moyenne du nombre de passagers
    partial_passengers_per_route = chunk_df.groupby('Itineraire Arrondissement')['passenger_count'].mean().reset_index()
    # Ajouter les résultats partiels à la liste
    partial_results.append(partial_passengers_per_route)

# Concaténer les résultats partiels en un seul DataFrame
passengers_per_route = pd.concat(partial_results)

# Regrouper à nouveau par Itineraire sur l'ensemble des données et calculer la moyenne finale du nombre de passagers
passengers_per_route = passengers_per_route.groupby('Itineraire Arrondissement')['passenger_count'].mean().reset_index()

# Trier les résultats par nombre moyen de passagers de manière décroissante
passengers_per_route = passengers_per_route.sort_values(by='passenger_count', ascending=False)

# Afficher le résultat
passengers_per_route

,Itineraire Arrondissement,passenger_count
15,EWR - Staten Island,4.000000
13,EWR - Manhattan,2.250000
12,EWR - EWR,2.222222
30,Staten Island - EWR,2.000000
18,Manhattan - EWR,1.840342
7,Brooklyn - EWR,1.800000
24,Queens - EWR,1.746377
27,Queens - Staten Island,1.649770
25,Queens - Manhattan,1.546327
20,Manhattan - Queens,1.521085


Ce résultat montre la moyenne du nombre de passagers pour différents itinéraires entre les arrondissements. Voici quelques interprétations possibles :

1. **Itinéraires avec le plus grand nombre moyen de passagers :**
   - "EWR - Staten Island" a la plus grande moyenne avec 4 passagers par trajet.
   - "EWR - Manhattan" suit avec une moyenne de 3 passagers par trajet.

2. **Itinéraires avec une moyenne modérée de passagers :**
   - "EWR - EWR" a une moyenne de 2.18 passagers par trajet.
   - "Staten Island - EWR" a une moyenne de 2 passagers par trajet.

3. **Itinéraires avec une moyenne plus basse de passagers :**
   - Les itinéraires reliant les aéroports aux autres arrondissements ont généralement des moyennes de passagers plus élevées.
   - Les itinéraires entre les arrondissements de Manhattan, Brooklyn et Queens ont des moyennes de passagers variables, allant d'environ 1.4 à 1.8 passagers par trajet.
   - Certains itinéraires, comme "Brooklyn - Brooklyn" et "Bronx - Bronx", ont des moyennes plus basses, autour de 1.0 à 1.3 passagers par trajet.

En résumé, cette analyse fournit des informations sur la densité de passagers pour différents itinéraires entre les arrondissements, ce qui peut être utile pour la planification des transports ou d'autres analyses liées aux déplacements dans la ville.

In [31]:
# Définir la taille du lot pour l'itération
chunk_size = 100000

# Initialiser une liste pour stocker les résultats partiels
partial_results = []

# Itérer sur les segments parquet par petits morceaux
for i in range(0, parquet_file.num_row_groups, chunk_size):
    # Lire un segment parquet
    chunk = parquet_file.read_row_group(i, columns=['puzone', 'dozone', 'passenger_count'])
    # Convertir le segment en DataFrame pandas
    chunk_df = chunk.to_pandas()
    # Créer une nouvelle colonne par concaténation de puzone et dozone pour chaque morceau
    chunk_df['Itineraire Zone'] = chunk_df.apply(lambda row: f"{row['puzone']} - {row['dozone']}", axis=1)
    # Regrouper par Itineraire pour chaque morceau et calculer la moyenne du nombre de passagers
    partial_passengers_per_route = chunk_df.groupby('Itineraire Zone')['passenger_count'].mean().reset_index()
    # Ajouter les résultats partiels à la liste
    partial_results.append(partial_passengers_per_route)

# Concaténer les résultats partiels en un seul DataFrame
passengers_per_route = pd.concat(partial_results)

# Regrouper à nouveau par Itineraire sur l'ensemble des données et calculer la moyenne finale du nombre de passagers
passengers_per_route = passengers_per_route.groupby('Itineraire Zone')['passenger_count'].mean().reset_index()

# Trier les résultats par nombre moyen de passagers de manière décroissante
passengers_per_route = passengers_per_route.sort_values(by='passenger_count', ascending=False)

# Afficher le résultat
passengers_per_route

,Itineraire Zone,passenger_count
11163,Red Hook - Laurelton,6.0
11726,SoHo - Mount Hope,6.0
11669,SoHo - Dyker Heights,6.0
9938,Morningside Heights - Flushing,6.0
907,Boerum Hill - Borough Park,6.0
...,...,...
3945,East Tremont - Pelham Bay Park,1.0
10286,Murray Hill - Sheepshead Bay,1.0
3946,East Tremont - Schuylerville/Edgewater Park,1.0
3947,East Tremont - Seaport,1.0


In [32]:
passengers_per_route.head(20)

,Itineraire Zone,passenger_count
11163,Red Hook - Laurelton,6.0
11726,SoHo - Mount Hope,6.0
11669,SoHo - Dyker Heights,6.0
9938,Morningside Heights - Flushing,6.0
907,Boerum Hill - Borough Park,6.0
2045,Clinton East - Bensonhurst West,6.0
14990,Williamsburg (North Side) - Jamaica Estates,6.0
10942,Prospect Park - Red Hook,6.0
11523,Seaport - Flushing Meadows-Corona Park,6.0
11522,Seaport - Flushing,6.0


Dans ces lignes du résultat, nous avons les itinéraires de zone avec le nombre moyen le plus élevé de passagers. Voici une explication pour chacun :

1. **Maspeth-Murray Hill (6.0 passagers en moyenne)** :
   - Cet itinéraire relie Maspeth à Murray Hill et a une moyenne de 6 passagers par trajet en moyenne. Cela indique une forte demande de transport sur cet itinéraire spécifique.

2. **Maspeth-Parkchester (6.0 passagers en moyenne)** :
   - Cet itinéraire relie Maspeth à Parkchester et a également une moyenne de 6 passagers par trajet en moyenne. Il semble y avoir une demande élevée de transport entre ces deux zones.

3. **Morningside Heights-Flushing (6.0 passagers en moyenne)** :
   - Cet itinéraire relie Morningside Heights à Flushing et affiche une moyenne de 6 passagers par trajet en moyenne. Cela suggère une forte activité de déplacement entre ces deux zones.

4. **Manhattanville-Greenwich Village North (6.0 passagers en moyenne)** :
   - Cet itinéraire relie Manhattanville à Greenwich Village North et a également une moyenne de 6 passagers par trajet en moyenne. Cela indique une demande importante de transport entre ces deux zones de Manhattan.

5. **Boerum Hill-Borough Park (6.0 passagers en moyenne)** :
   - Cet itinéraire relie Boerum Hill à Borough Park et a une moyenne de 6 passagers par trajet en moyenne. Cela suggère une forte demande de déplacement entre ces deux zones.

En résumé, ces lignes indiquent les itinéraires de zone avec la plus forte moyenne de passagers par trajet, ce qui peut être utile pour la planification des transports et la gestion des ressources.

In [33]:
passengers_per_route.passenger_count.unique()

array([6.        , 5.        , 4.5       , ..., 1.03333333, 1.025     ,
       1.        ])

In [34]:
passengers_per_route.tail (20)

,Itineraire Zone,passenger_count
3933,East Tremont - East Chelsea,1.0
3934,East Tremont - East Harlem South,1.0
3935,East Tremont - East New York/Pennsylvania Avenue,1.0
10300,Murray Hill - Sunset Park East,1.0
3936,East Tremont - East Tremont,1.0
3938,East Tremont - Financial District South,1.0
10283,Murray Hill - Saint Michaels Cemetery/Woodside,1.0
3939,East Tremont - Longwood,1.0
10295,Murray Hill - Starrett City,1.0
3940,East Tremont - Morningside Heights,1.0


### Tarif moyen par itineraire de zone

In [36]:
# Définir la taille du lot pour l'itération
chunk_size = 100000

# Lire le fichier parquet et obtenir les métadonnées
parquet_file = pq.ParquetFile('trips_cleaned.parquet')

# Initialiser une liste pour stocker les résultats partiels
partial_results = []

# Itérer sur les segments parquet par petits morceaux
for i in range(0, parquet_file.num_row_groups, chunk_size):
    # Lire un segment parquet
    chunk = parquet_file.read_row_group(i, columns=['puzone', 'dozone', 'total_amount'])
    # Convertir le segment en DataFrame pandas
    chunk_df = chunk.to_pandas()
    # Créer une nouvelle colonne par concaténation de puzone et dozone pour chaque morceau
    chunk_df['Itineraire Zone'] = chunk_df.apply(lambda row: f"{row['puzone']} TO {row['dozone']}", axis=1)
    # Calculer le tarif moyen par itinéraire de zone pour chaque morceau
    partial_fare_per_route = chunk_df.groupby('Itineraire Zone')['total_amount'].mean().reset_index()
    # Ajouter les résultats partiels à la liste
    partial_results.append(partial_fare_per_route)

# Concaténer les résultats partiels en un seul DataFrame
fare_per_route = pd.concat(partial_results)

# Regrouper à nouveau par itinéraire de zone sur l'ensemble des données et calculer le tarif moyen final
fare_per_route = fare_per_route.groupby('Itineraire Zone')['total_amount'].mean().reset_index()

# Trier les résultats par tarif moyen de manière décroissante
fare_per_route = fare_per_route.sort_values(by='total_amount', ascending=False)

# Afficher le résultat
fare_per_route.head(20)

,Itineraire Zone,total_amount
6865,Kensington TO Downtown Brooklyn/MetroTech,369.000000
9124,Midtown Center TO Bloomfield/Emerson Hill,236.500000
12081,Springfield Gardens South TO Newark Airport,210.646667
10080,Mott Haven/Port Morris TO Lower East Side,201.000000
6866,Kensington TO East Flatbush/Farragut,201.000000
6583,JFK Airport TO Newark Airport,198.016793
10351,Newark Airport TO JFK Airport,196.690000
285,Baisley Park TO Newark Airport,196.385000
15007,Williamsburg (North Side) TO Newark Airport,194.350000
3400,East Elmhurst TO Newark Airport,185.543333


Ce résultat est une liste d'itinéraires avec le montant total moyen facturé pour chaque trajet. Chaque ligne représente un itinéraire spécifique, avec le nom de la zone de départ et d'arrivée, ainsi que le montant total facturé pour ce trajet.

- **Itinéraire Zone**: C'est la description de l'itinéraire, indiquant la zone de départ et la zone d'arrivée.

- **total_amount**: C'est le montant total facturé pour ce trajet spécifique.

Par exemple, la première ligne indique qu?un trajet de Kensington à Downtown Brooklyn/MetroTech coute en moyenne 369.00$ au passager. La deuxième ligne montre qu'un trajet de Midtown Center à Bloomfield/Emerson Hill coute en moyenne 236.50$, et ainsi de suite.

Ces données pourraient être utilisées pour comprendre les tarifs moyens pour différents trajets ou itinéraires spécifiques, ce qui pourrait être utile pour les analyses tarifaires ou pour identifier des itinéraires coûteux ou populaires.

In [39]:
# Tracer le graphique à barres horizontales
fig = px.bar(fare_per_route.head(10), x='total_amount', y='Itineraire Zone', orientation='h', 
             title='Top 10 des itineraires les plus chers', labels={'total_amount':'Montant total', 'Itineraire Zone':'Itinéraire'})
fig.show()

In [40]:
fig = px.bar(fare_per_route.tail(10), x='total_amount', y='Itineraire Zone', orientation='h', 
             title='Top 10 des itineraires les moins chers', labels={'total_amount':'Montant total', 'Itineraire Zone':'Itinéraire'})
fig.show()

In [41]:
fare_per_route.tail(10)

,Itineraire Zone,total_amount
2992,Dyker Heights TO Bay Ridge,5.20
5004,Flushing TO Flushing Meadows-Corona Park,4.50
11948,South Ozone Park TO Marine Park/Floyd Bennett ...,4.50
10411,Oakland Gardens TO Fresh Meadows,4.50
11438,Saint Michaels Cemetery/Woodside TO JFK Airport,4.50
11384,Rosedale TO Rosedale,4.50
1508,Central Harlem North TO Crotona Park,1.01
6806,Jamaica TO Flushing Meadows-Corona Park,1.01
2881,Douglaston TO Douglaston,1.00
12171,Steinway TO Astoria Park,1.00


Ce résultat semble montrer les itinéraires avec les montants les plus bas. Voici quelques commentaires sur certains des itinéraires affichés :

1. **Dyker Heights TO Bay Ridge**: Cet itinéraire a un montant total de 5,20. Il semble être un court trajet entre deux quartiers voisins.

2. **Flushing TO Flushing Meadows-Corona Park**: Avec un montant total de 4,50, cet itinéraire semble également être un court trajet dans le même quartier.

3. **South Ozone Park TO Marine Park/Floyd Bennett ...**: Cet itinéraire a également un montant total de 4,50. Il pourrait s'agir d'un autre court trajet entre deux quartiers voisins.

4. **Central Harlem North TO Crotona Park**: Avec un montant total de 1,01, cet itinéraire semble être un court trajet à l'intérieur du même quartier ou entre des quartiers voisins.

5. **Jamaica TO Flushing Meadows-Corona Park**: Cet itinéraire a également un montant total de 1,01. Il semble également être un court trajet à l'intérieur du même quartier ou entre des quartiers voisins.

Ces montants très bas pourraient indiquer des trajets très courts ou des erreurs de saisie de données, car les montants sont bien inférieurs à ce qui est généralement observé pour les trajets en taxi.